In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import lasio
import os

## Сохранение файлов continuous и discrete в формате .csv

In [4]:
def get_discrete_files(path):
    files = os.listdir(path)
    return [file for file in files if file.endswith('_discrete.las')]

In [5]:
def get_continuous_files(path):
    files = os.listdir(path)
    return [file for file in files if file.endswith('_continuous.las')]

In [6]:
def las_to_csv(las_file, output_path):
    las = lasio.read(las_file)
    df = las.df()
    df.reset_index(inplace=True)
    df.to_csv(f"{output_path}/{las_file.split('/')[-1].replace('.las', '.csv')}", index=False)

In [84]:
las_folder_path = '<your_path_here>'

In [6]:
discrete_files = get_discrete_files(las_folder_path)
continuous_files = get_continuous_files(las_folder_path)

In [7]:
dicrete_output_path = '<your_path_here>'
continuous_output_path = '<your_path_here>'

In [9]:
os.makedirs(dicrete_output_path, exist_ok=True)
os.makedirs(continuous_output_path, exist_ok=True)

In [10]:
for file in discrete_files:
    las_to_csv(f"{las_folder_path}/{file}", dicrete_output_path)

In [11]:
for file in continuous_files:
    las_to_csv(f"{las_folder_path}/{file}", continuous_output_path)

## Создание датасета, состоящего из обобщенных данных из файлов continuous и discrete

In [2]:
discrete_files_path = '<your_path_here>'
continuous_files_path = '<your_path_here>'

In [ ]:
# создаем список для хранения данных по скважинам из файлов discrete

wells_discrete = [["WELL", "DEPT", "SSTVD", "Z", "FACIES", "LOG9"]]

for file in os.listdir(discrete_files_path):
    if file.endswith('.csv'):
        file_path = os.path.join(discrete_files_path, file)
        well_name = file.split('_')[0]
        with open(file_path) as f:
            lines = f.readlines()
            for line in lines[1:]:
                parsed_line = []
                for i in line.split(','):
                    try:
                        parsed_line.append(float(i.strip()))
                    except:
                        parsed_line.append(None)
                wells_discrete.append([well_name] + parsed_line)

In [5]:
# создаем список для хранения данных по скважинам из файлов continuous
wells_continuous = [["WELL", "DEPT", "SSTVD", "Z", "NEU", "GGKP"]]

for file in os.listdir(continuous_files_path):
    if file.endswith('.csv'):
        file_path = os.path.join(continuous_files_path, file)
        well_name = file.split('_')[0]
        with open(file_path) as f:
            lines = f.readlines()
            for line in lines[1:]:
                parsed_line = []
                for i in line.split(','):
                    try:
                        parsed_line.append(float(i.strip()))
                    except ValueError:
                        parsed_line.append(None)
                wells_continuous.append([well_name] + parsed_line)

In [7]:
# создаем датафреймы для дальнейшей работы
discrete_df =  pd.DataFrame(wells_discrete[1:], columns=wells_discrete[0])

In [8]:
# удаляем строки с пустыми значениями в столбцах FACIES и LOG9
discrete_df.dropna(subset=['FACIES', 'LOG9'], how='all', inplace=True)

In [9]:
# удаляем столбцы SSTVD и Z
discrete_df.drop(columns=['SSTVD', 'Z'], inplace=True)

In [28]:
continuous_df = pd.DataFrame(wells_continuous[1:], columns=wells_continuous[0])

In [30]:
# удаляем строки с пустыми значениями в столбцах NEU и GGKP
continuous_df.dropna(subset=['NEU', 'GGKP'], how='all', inplace=True)

In [32]:
# сортируем датафреймы по столбцам WELL и DEPT
continuous_df.sort_values(by=['WELL', 'DEPT'], inplace=True)
discrete_df.sort_values(by=['WELL', 'DEPT'], inplace=True)

In [7]:
# Импортируем библиотеку для построения KDTree
from scipy.spatial import cKDTree

def nearest_merge_with_threshold(left_df, right_df, on, max_distance, suffixes=('_left', '_right')):
    # Эта функция добавляет порог для максимального допустимого расстояния между значениями DEPT
    merged_df = pd.DataFrame() # Инициализируем пустой DataFrame для хранения результата

    # Обработка каждой скважины отдельно
    for well in pd.unique(left_df[on[0]]):
        left_subset = left_df[left_df[on[0]] == well].copy()
        right_subset = right_df[right_df[on[0]] == well].copy()

        if not left_subset.empty and not right_subset.empty:
            # Строим KDTree с использованием DEPT из правого поднабора
            kdtree = cKDTree(right_subset[[on[1]]])

            # Получаем расстояния и индексы ближайших точек
            distances, indices = kdtree.query(left_subset[[on[1]]], distance_upper_bound=max_distance)

            # Добавляем индексы и расстояния в левый поднабор
            left_subset['right_index'] = indices
            left_subset['distances'] = distances

            # Применяем порог к расстояниям
            mask = left_subset['distances'] <= max_distance
            left_subset.loc[~mask, 'right_index'] = np.nan # Присваиваем NaN, если расстояние слишком велико

            # Совмещаем результаты с правым поднабором на основе найденных индексов
            left_subset = left_subset.merge(right_subset.reset_index(), left_on='right_index', right_index=True, how='left', suffixes=suffixes)
            merged_df = pd.concat([merged_df, left_subset], ignore_index=True)

    # Удаляем лишние столбцы
    merged_df.drop(columns=['right_index', 'distances'], inplace=True)
    return merged_df


In [34]:
# Определяем столбцы для объединения и пороговое максимальное расстояние
on_columns = ['WELL', 'DEPT']
max_distance = 0.1 # Определяем максимальное допустимое расстояние между значениями DEPT

In [35]:
# Выполняем слияние с порогом расстояния
result_df = nearest_merge_with_threshold(continuous_df, discrete_df, on=on_columns, max_distance=max_distance)

In [36]:
result_df.head()

,WELL_left,DEPT_left,SSTVD,Z,NEU,GGKP,index,WELL_right,DEPT_right,FACIES,LOG9
0,12,2738.4756,1952.051221,-1952.051221,0.3154,NaN,NaN,NaN,NaN,NaN,NaN
1,12,2738.6280,1952.155861,-1952.155861,0.3194,NaN,NaN,NaN,NaN,NaN,NaN
2,12,2738.7804,1952.260501,-1952.260501,0.3239,NaN,NaN,NaN,NaN,NaN,NaN
3,12,2738.9328,1952.365140,-1952.365140,0.3271,NaN,NaN,NaN,NaN,NaN,NaN
4,12,2739.0852,1952.469780,-1952.469780,0.3286,NaN,NaN,NaN,NaN,NaN,NaN


In [37]:
# Удаляем лишние столбцы
result_df.drop(columns=['WELL_right', 'DEPT_right', 'index'], inplace=True)

In [38]:
result_df.head()

,WELL_left,DEPT_left,SSTVD,Z,NEU,GGKP,FACIES,LOG9
0,12,2738.4756,1952.051221,-1952.051221,0.3154,NaN,NaN,NaN
1,12,2738.6280,1952.155861,-1952.155861,0.3194,NaN,NaN,NaN
2,12,2738.7804,1952.260501,-1952.260501,0.3239,NaN,NaN,NaN
3,12,2738.9328,1952.365140,-1952.365140,0.3271,NaN,NaN,NaN
4,12,2739.0852,1952.469780,-1952.469780,0.3286,NaN,NaN,NaN


In [39]:
# Переименовываем столбцы
result_df.columns = ['WELL', 'DEPT', 'SSTVD', 'Z', 'NEU', 'GGKP', 'FACIES', 'LOG9']

In [40]:
result_df

,WELL,DEPT,SSTVD,Z,NEU,GGKP,FACIES,LOG9
0,12,2738.4756,1952.051221,-1952.051221,0.3154,NaN,NaN,NaN
1,12,2738.6280,1952.155861,-1952.155861,0.3194,NaN,NaN,NaN
2,12,2738.7804,1952.260501,-1952.260501,0.3239,NaN,NaN,NaN
3,12,2738.9328,1952.365140,-1952.365140,0.3271,NaN,NaN,NaN
4,12,2739.0852,1952.469780,-1952.469780,0.3286,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...
264738,K-66,2833.5732,2210.045565,-2210.045565,NaN,2.59,NaN,NaN
264739,K-66,2833.7256,2210.191964,-2210.191964,NaN,2.60,NaN,NaN
264740,K-66,2833.8780,2210.338363,-2210.338363,NaN,2.60,NaN,NaN
264741,K-66,2834.0304,2210.484762,-2210.484762,NaN,2.60,NaN,NaN


In [41]:
# Сохраняем результат в CSV
result_df.to_csv('<your_path_here>', index=False)

## Использовнаие алгоритма RandomForest для предсказания значения FACIES

In [2]:
df = pd.read_csv('continuous_discrete_merged.csv')

/tmp/ipykernel_54989/2764342425.py:1: DtypeWarning: Columns (0) have mixed types. Specify dtype option on import or set low_memory=False.
  df = pd.read_csv('continuous_discrete_merged.csv')


In [3]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 264743 entries, 0 to 264742
Data columns (total 8 columns):
 #   Column  Non-Null Count   Dtype  
---  ------  --------------   -----  
 0   WELL    264743 non-null  object 
 1   DEPT    264743 non-null  float64
 2   SSTVD   264743 non-null  float64
 3   Z       264743 non-null  float64
 4   NEU     254752 non-null  float64
 5   GGKP    210100 non-null  float64
 6   FACIES  2751 non-null    float64
 7   LOG9    2916 non-null    float64
dtypes: float64(7), object(1)
memory usage: 16.2+ MB


In [4]:
# Удаляем строки с пустыми значениями в столбцах NEU и GGKP
df.dropna(subset=['NEU', 'GGKP'], inplace=True)

In [5]:
df.info()

<class 'pandas.core.frame.DataFrame'>
Index: 200109 entries, 18 to 264715
Data columns (total 8 columns):
 #   Column  Non-Null Count   Dtype  
---  ------  --------------   -----  
 0   WELL    200109 non-null  object 
 1   DEPT    200109 non-null  float64
 2   SSTVD   200109 non-null  float64
 3   Z       200109 non-null  float64
 4   NEU     200109 non-null  float64
 5   GGKP    200109 non-null  float64
 6   FACIES  2646 non-null    float64
 7   LOG9    2811 non-null    float64
dtypes: float64(7), object(1)
memory usage: 13.7+ MB


In [6]:
# Удаляем строки с пустыми значениями в столбцах FACIES и LOG9
df_filtered = df.dropna(subset=['FACIES', 'LOG9'])

In [7]:
# Определяем признаки и целевую переменную
X = df_filtered[[col for col in df_filtered.columns if col.startswith('DEPT') or col.startswith('GGKP')]]
y = df_filtered['FACIES'] # Измените на 'LOG9', если хотите предсказать LOG9

In [8]:
# Импортируем библиотеки для обучения модели
from sklearn.ensemble import RandomForestClassifier
from sklearn.model_selection import train_test_split
from sklearn.metrics import classification_report
from sklearn.model_selection import RandomizedSearchCV, GridSearchCV
from sklearn.model_selection import StratifiedKFold

In [9]:
# Разделяем данные на обучающую и тестовую выборки
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=1)

In [10]:
# Random Search для исследования пространства параметров
param_dist = {
    'n_estimators': [100, 200, 300, 400, 500, 600],
    'criterion' : ['gini', 'entropy', 'log_loss'], 
    'min_samples_leaf': [2, 3, 4],
    'max_depth': [5, 10, 15, 20, 25]
}
cv = StratifiedKFold(n_splits=2) # Используем Stratified K-Fold для сохранения баланса классов
rf_random_search = RandomizedSearchCV(RandomForestClassifier(), param_distributions=param_dist, n_iter=50, cv=cv, n_jobs=-1) # n_iter - количество случайных комбинаций параметров
rf_random_search.fit(X_train, y_train)

# Лучшие параметры после Random Search
param_grid = {
    'n_estimators': [rf_random_search.best_params_['n_estimators'] - 50, rf_random_search.best_params_['n_estimators'], rf_random_search.best_params_['n_estimators'] + 50],
    'criterion': [rf_random_search.best_params_['criterion']],
    'min_samples_leaf': [rf_random_search.best_params_['min_samples_leaf'] - 1, rf_random_search.best_params_['min_samples_leaf'], rf_random_search.best_params_['min_samples_leaf'] + 1],
    'max_depth': [rf_random_search.best_params_['max_depth'] - 5, rf_random_search.best_params_['max_depth'], rf_random_search.best_params_['max_depth'] + 5]
}

# Grid Search для более точного поиска
rf_grid_search = GridSearchCV(RandomForestClassifier(), param_grid=param_grid, cv=cv, n_jobs=-1)
rf_grid_search.fit(X_train, y_train)

# Лучшие параметры после Grid Search
best_params = rf_grid_search.best_params_

In [11]:
# Создание модели с лучшими параметрами
best_rf_model = RandomForestClassifier(**best_params)

# Обучение модели на всем тренировочном наборе данных
best_rf_model.fit(X_train, y_train)

RandomForestClassifier(criterion='log_loss', max_depth=25, min_samples_leaf=2,
                       n_estimators=300)

In [12]:
# Предсказание на тестовом наборе данных
y_pred = rf_grid_search.predict(X_test)

In [13]:
# Оценка модели
print(classification_report(y_test, y_pred))

              precision    recall  f1-score   support

         1.0       0.48      0.53      0.50        87
         2.0       0.39      0.32      0.35        56
         3.0       0.56      0.65      0.60        75
         4.0       0.61      0.59      0.60        46
         5.0       0.00      0.00      0.00         0
         6.0       0.00      0.00      0.00         1
         7.0       0.50      0.22      0.31         9
         8.0       0.43      0.45      0.44        44
         9.0       0.40      0.22      0.29         9
        10.0       0.57      0.46      0.51        26
        11.0       0.69      0.76      0.72        98
        12.0       0.71      0.45      0.56        11
        13.0       0.61      0.61      0.61        18
        15.0       0.54      0.51      0.53        49

    accuracy                           0.55       529
   macro avg       0.46      0.41      0.43       529
weighted avg       0.55      0.55      0.54       529



/home/user/projects/hakaton/venv/lib/python3.10/site-packages/sklearn/metrics/_classification.py:1509: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
/home/user/projects/hakaton/venv/lib/python3.10/site-packages/sklearn/metrics/_classification.py:1509: UndefinedMetricWarning: Recall is ill-defined and being set to 0.0 in labels with no true samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
/home/user/projects/hakaton/venv/lib/python3.10/site-packages/sklearn/metrics/_classification.py:1509: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(

In [16]:
# Предсказание на всем наборе данных
y_pred_all = rf_grid_search.predict(X)

In [20]:
# Добавляем предсказанные значения в датафрейм в колонку FACIES_PRED
df_filtered['FACIES_PRED'] = y_pred_all

/tmp/ipykernel_54989/2639750789.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_filtered['FACIES_PRED'] = y_pred_all


In [31]:
df_filtered.head()

,WELL,DEPT,SSTVD,Z,NEU,GGKP,FACIES,LOG9,FACIES_PRED
1464,12,2961.5892,2106.884575,-2106.884575,0.3395,2.59,8.0,7.0,8.0
1465,12,2961.7416,2106.991764,-2106.991764,0.2803,2.57,8.0,7.0,3.0
1575,12,2978.5056,2118.848924,-2118.848924,0.2246,2.39,7.0,7.0,7.0
1606,12,2983.2300,2122.204023,-2122.204023,0.2397,2.52,2.0,2.0,2.0
1621,12,2985.5160,2123.828328,-2123.828328,0.2057,2.40,15.0,2.0,3.0


In [21]:
# Сохраняем результат в CSV
df_filtered.to_csv('facies_predictions.csv', index=False)

## Атоматизация процесса разметки файлов continuous по классам FACIES с использованием значений DEPT и GGKP с помощью алгоритма RandomForest

In [36]:
# Путь к файлу с данными continuous
continuous_file_path = '<<your_path_here>>'

In [26]:
# Чтение данных из файла и преобразование в DataFrame
las = lasio.read(continuous_file_path)
df = las.df()
df.reset_index(inplace=True)

In [27]:
df.head()

,DEPT,SSTVD,Z,NEU,GGKP
0,0.0000,-82.2000,82.2000,NaN,NaN
1,0.1524,-82.0476,82.0476,NaN,NaN
2,0.3048,-81.8952,81.8952,NaN,NaN
3,0.4572,-81.7428,81.7428,NaN,NaN
4,0.6096,-81.5904,81.5904,NaN,NaN


In [28]:
# Удаляем строки с пустыми значениями в столбцах GGKP
df.dropna(subset=['GGKP'], inplace=True)

In [30]:
df.head()

,DEPT,SSTVD,Z,NEU,GGKP
17987,2741.2188,1953.934735,-1953.934735,0.2911,2.44
17988,2741.3712,1954.039375,-1954.039375,0.3014,2.53
17989,2741.5236,1954.144015,-1954.144015,0.3325,2.44
17990,2741.6760,1954.248654,-1954.248654,0.3527,2.43
17991,2741.8284,1954.353294,-1954.353294,0.3553,2.40


In [32]:
# Определяем признаки для предсказания
X = df[[col for col in df.columns if col.startswith('DEPT') or col.startswith('GGKP')]]

In [33]:
# Предсказание классов FACIES с использованием модели RandomForest
facies_predictions = rf_grid_search.predict(X)

In [34]:
#  Добавляем предсказанные значения в датафрейм в колонку FACIES_PRED
df['FACIES_PRED'] = facies_predictions

In [35]:
df.head()

,DEPT,SSTVD,Z,NEU,GGKP,FACIES_PRED
17987,2741.2188,1953.934735,-1953.934735,0.2911,2.44,1.0
17988,2741.3712,1954.039375,-1954.039375,0.3014,2.53,1.0
17989,2741.5236,1954.144015,-1954.144015,0.3325,2.44,1.0
17990,2741.6760,1954.248654,-1954.248654,0.3527,2.43,11.0
17991,2741.8284,1954.353294,-1954.353294,0.3553,2.40,11.0
